# Edit

In [1]:
from nnsight import LanguageModel
from nnsight.edit import Edit
import torch

/share/u/caden/.conda/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Load some dispatched model

In [2]:
model = LanguageModel("openai-community/gpt2", device_map="cuda:0", dispatch=True)

Create a simple wrapper module that passes its args through. Useful for hooking the forward pass.

In [3]:
class WrapperModule(torch.nn.Module):

    def forward(self, *args, **kwargs):
        if len(args) == 1:
            args = args[0]
        
        return args

Declare an edit.

In [4]:
edit_one = Edit(
    "transformer.h.0.attn", # Path to module that we want to edit
    "query", # Name of parameter we want to wrap
    "query_wrapper", # Name of wrapper in our module
    WrapperModule() # Module we'd like to wrap with
)

We could also modify the args as they are passed through.

In [5]:
class EditModule(torch.nn.Module):

    def forward(self, *args, **kwargs):
        if len(args) == 1:
            args = args[0]
        
        value = args * 100

        return value

edit_two = Edit(
    "transformer.h.0.attn", # Path to module that we want to edit
    "attn_weights", # Name of parameter we want to wrap
    "weights_wrapper", # Name of wrapper in our module
    WrapperModule() # Module we'd like to wrap with
)

We can use various utils from edit module to see how Dynamo will transform bytecode.

In [6]:
from nnsight.edit import print_gm

with model.trace(" "):
    mlp_input = model.transformer.h[0].mlp.input.save()

print_gm(model.transformer.h[0].mlp, mlp_input[0][0])

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


GraphModule(
  (L__self___dropout): Dropout(p=0.1, inplace=False)
)
def forward(self, L_hidden_states_ : torch.Tensor):
    l_hidden_states_ = L_hidden_states_
    size = l_hidden_states_.size()
    l__self___c_fc_bias = self.L__self___c_fc_bias
    size_1 = l_hidden_states_.size(-1)
    view = l_hidden_states_.view(-1, size_1);  l_hidden_states_ = size_1 = None
    l__self___c_fc_weight = self.L__self___c_fc_weight
    x = torch.addmm(l__self___c_fc_bias, view, l__self___c_fc_weight);  l__self___c_fc_bias = view = 
l__self___c_fc_weight = None
    hidden_states = x.view((1, 1, 3072));  x = None
    mul = 0.5 * hidden_states
    pow_1 = torch.pow(hidden_states, 3.0)
    mul_1 = 0.044715 * pow_1;  pow_1 = None
    add = hidden_states + mul_1;  hidden_states = mul_1 = None
    mul_2 = 0.7978845608028654 * add;  add = None
    tanh = torch.tanh(mul_2);  mul_2 = None
    add_1 = 1.0 + tanh;  tanh = None
    hidden_states_1 = mul * add_1;  mul = add_1 = None
    l__self___c_proj_bias = self.L__self___c_proj_bias
    view_2 = hidden_states_1.view(-1, 3072);  hidden_states_1 = None
    l__self___c_proj_weight = self.L__self___c_proj_weight
    x_2 = torch.addmm(l__self___c_proj_bias, view_2, l__self___c_proj_weight);  l__self___c_proj_bias = view_2 = 
l__self___c_proj_weight = None
    hidden_states_2 = x_2.view((1, 1, 768));  x_2 = None
    hidden_states_3 = self.L__self___dropout(hidden_states_2);  hidden_states_2 = None
    return (hidden_states_3,)

We'll load the edits into the module, and access at the points we declared.

In [7]:
model.load_edits([edit_one, edit_two])

Note that the first time running a model after loading edits will take a little longer.

In [8]:
with model.trace("National Deep Inferencing Fabric", scan=False, validate=False):
    value_one = model.transformer.h[0].attn.query_wrapper.output.save()
    value_two = model.transformer.h[0].attn.weights_wrapper.input.save()

print(value_one.shape)
print(value_two.shape)

torch.Size([1, 6, 768])
((torch.Size([1, 12, 6, 6]),), {})


You can edit and intervene as normal.

In [ ]:
with model.trace("National Deep Inferencing Fabric", scan=False, validate=False):
    value_one = model.transformer.h[0].attn.query_wrapper.output.grad.save()
    value_two = model.transformer.h[0].attn.weights_wrapper.output * 100

    value = model.output.logits.sum()
    value.backward()

print(value_one.shape)
print(value_two.shape)

# Technical Notes

Edit has an option kwarg, "instance", which takes the instance at which you want to intervene. 

When Dynamo has a graph break, this makes it so that variable names are reset. By default, the edit module grabs the value from the first graph it is found in.

In [10]:
edit_one = Edit(
    "transformer.h.0.attn", # Path to module that we want to edit
    "query", # Name of parameter we want to wrap
    "query_wrapper", # Name of wrapper in our module
    WrapperModule() # Module we'd like to wrap with
)